<a href="https://colab.research.google.com/github/SharmaAshwini/UNT/blob/main/Methods%20in%20Empirical%20Analysis/Exercise_9_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ICE-9** Cross Validation

###**INSTRUCTIONS**
#####Provide your observation/summary for all tutorials and tasks. Marks will be deducted for those who dont write the summary.
#####Print this file as pdf and submit both pdf and .ipynb file(No screenshots). Not submitting any of these files will impact their grades.

###Tutorial

In [ ]:
!gdown '1hH9SJLOwZKNrzHhqGgFD4NmE9JDx2mYx'

Downloading...
From: https://drive.google.com/uc?id=1hH9SJLOwZKNrzHhqGgFD4NmE9JDx2mYx
To: /content/income_evaluation.csv
100% 3.81M/3.81M [00:00<00:00, 70.4MB/s]


####Performing EDA on the dataset

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, LeaveOneOut,\
RepeatedKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv('income_evaluation.csv', na_values=' ?')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
df.isna().sum()

age                   0
 workclass         1836
 fnlwgt               0
 education            0
 education-num        0
 marital-status       0
 occupation        1843
 relationship         0
 race                 0
 sex                  0
 capital-gain         0
 capital-loss         0
 hours-per-week       0
 native-country     583
 income               0
dtype: int64

In [ ]:
df.fillna('missing', inplace=True)

In [ ]:
df.isna().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [ ]:
X = df.drop(' income', axis=1)
y = df[' income']

In [ ]:
X.shape

(32561, 14)

In [ ]:
y.value_counts()

 <=50K    24720
 >50K      7841
Name:  income, dtype: int64

###1. KFold Cross Validation

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
32561/5

6512.2

In [ ]:
6512*4

26048

In [ ]:
i = 1
for train_set, test_set in kf.split(X=X):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print("-------------------------")
    i += 1

iteration  1
[ 6513  6514  6515 ... 32558 32559 32560]  having : 26048
[   0    1    2 ... 6510 6511 6512]  having : 6513
-------------------------
iteration  2
[    0     1     2 ... 32558 32559 32560]  having : 26049
[ 6513  6514  6515 ... 13022 13023 13024]  having : 6512
-------------------------
iteration  3
[    0     1     2 ... 32558 32559 32560]  having : 26049
[13025 13026 13027 ... 19534 19535 19536]  having : 6512
-------------------------
iteration  4
[    0     1     2 ... 32558 32559 32560]  having : 26049
[19537 19538 19539 ... 26046 26047 26048]  having : 6512
-------------------------
iteration  5
[    0     1     2 ... 26046 26047 26048]  having : 26049
[26049 26050 26051 ... 32558 32559 32560]  having : 6512
-------------------------


In [ ]:
num_cols = X.select_dtypes(include=np.number).columns
num_cols

Index(['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss',
       ' hours-per-week'],
      dtype='object')

In [ ]:
cat_cols = X.select_dtypes(exclude=np.number).columns
cat_cols

Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country'],
      dtype='object')

In [ ]:
ct = ColumnTransformer([
    ('rob', RobustScaler(), num_cols),
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols)
])

In [ ]:
pipe = Pipeline([
    ('ct_step', ct),
    ('model', RandomForestClassifier(n_estimators=10, random_state=0))
])

In [ ]:
X.loc[6513:].head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
6513,29,Private,280344,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States
6514,45,Private,202496,Bachelors,13,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,37,United-States
6515,61,Self-emp-inc,134768,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States
6516,40,Private,175686,Some-college,10,Divorced,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
6517,24,Private,194748,HS-grad,9,Never-married,Transport-moving,Not-in-family,White,Female,0,0,49,United-States


In [ ]:
scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


In [ ]:
np.array(scores)

array([0.84784278, 0.84520885, 0.84613022, 0.84858722, 0.85165848])

In [ ]:
np.array(scores).mean()

0.8478855085142512

In [ ]:
np.array(scores).std()

0.0022349531977626388

###2. Stratified KFold Cross Validation

In [ ]:
y.value_counts()

 <=50K    24720
 >50K      7841
Name:  income, dtype: int64

In [ ]:
7841/5

1568.2

In [ ]:
24720/5

4944.0

In [ ]:
4944*4

19776

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
scores_skf = []
i = 1
for train_set, test_set in skf.split(X, y):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores_skf.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


In [ ]:
scores_skf

[0.8473821587594043,
 0.8432125307125307,
 0.8421375921375921,
 0.8425982800982801,
 0.8536547911547911]

In [ ]:
i = 1
for train_set, test_set in skf.split(X=X, y=y):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print()
    print("y train counts: \n", y[train_set].value_counts())
    print("y test counts: \n", y[test_set].value_counts())
    print("-------------------------")
    i += 1

iteration  1
[ 6499  6500  6512 ... 32558 32559 32560]  having : 26048
[   0    1    2 ... 6514 6515 6516]  having : 6513

y train counts: 
  <=50K    19776
 >50K      6272
Name:  income, dtype: int64
y test counts: 
  <=50K    4944
 >50K     1569
Name:  income, dtype: int64
-------------------------
iteration  2
[    0     1     2 ... 32558 32559 32560]  having : 26049
[ 6499  6500  6512 ... 13121 13123 13125]  having : 6512

y train counts: 
  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts: 
  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
-------------------------
iteration  3
[    0     1     2 ... 32558 32559 32560]  having : 26049
[12997 12999 13000 ... 19727 19729 19733]  having : 6512

y train counts: 
  <=50K    19776
 >50K      6273
Name:  income, dtype: int64
y test counts: 
  <=50K    4944
 >50K     1568
Name:  income, dtype: int64
-------------------------
iteration  4
[    0     1     2 ... 32558 32559 32560]  having : 26049
[19482 1948

In [ ]:
result_kf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=5)

In [ ]:
result_kf

array([0.84738216, 0.84321253, 0.84213759, 0.84259828, 0.85365479])

###3. Leave one out Cross Validation

In [ ]:
start = time.time()
result_kf10 = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=KFold(n_splits=10))
result_kf10
print("time taken: ", time.time()-start)

time taken:  5.636351108551025


In [ ]:
result_kf10

array([0.83880872, 0.85165848, 0.84981572, 0.84367322, 0.85135135,
       0.84613022, 0.84520885, 0.84797297, 0.8544226 , 0.84459459])

In [ ]:
start = time.time()
result_loocv = cross_val_score(estimator=pipe, X=X.head(100), y=y.head(100),
                               scoring='accuracy', cv=LeaveOneOut())
print("time taken: ", time.time()-start)

time taken:  3.4439473152160645


In [ ]:
32531/100

325.31

In [ ]:
325*8

2600

In [ ]:
325*8/60

43.333333333333336

In [ ]:
result_loocv

array([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.])

In [ ]:
result_loocv.mean()

0.8

###4. Repeated KFold Cross Validation

In [ ]:
start = time.time()
result_rkf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy',
                              cv=RepeatedKFold(n_splits=5, n_repeats=5))
result_rkf
print("time taken: ", time.time()-start)

time taken:  14.659301042556763


In [ ]:
result_rkf

array([0.85337018, 0.84874079, 0.84674447, 0.84336609, 0.8495086 ,
       0.85521265, 0.84643735, 0.84920147, 0.84305897, 0.85135135,
       0.84692154, 0.84520885, 0.85150491, 0.84474816, 0.85242629,
       0.84385076, 0.85165848, 0.84136978, 0.84689803, 0.85304054,
       0.84538615, 0.85488329, 0.85027641, 0.83952703, 0.85626536])

###5. Performing Cross validation on a training set, and then verifying our results on a seperate testset to generalize our results.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
start = time.time()
result_tts = cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='accuracy', cv=KFold(n_splits=5))
print("time taken: ", time.time()-start)

time taken:  2.8047659397125244


In [ ]:
result_tts

array([0.85547025, 0.84702495, 0.84184261, 0.84661163, 0.84968324])

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('ct_step',
                 ColumnTransformer(transformers=[('rob', RobustScaler(),
                                                  Index(['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss',
       ' hours-per-week'],
      dtype='object')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index([' workclass', ' education', ' marital-status', ' occupation',
       ' relationship', ' race', ' sex', ' native-country'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(n_estimators=10, random_state=0))])

In [ ]:
pipe.score(X_test, y_test)

0.8446184553968985

In [ ]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='kemfi', cv=KFold(n_splits=5))

ValueError: ignored

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [ ]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='roc_auc', cv=KFold(n_splits=5))

array([0.88177155, 0.8888249 , 0.87771091, 0.87869971, 0.87865483])

In [ ]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='f1_macro', cv=KFold(n_splits=5))

array([0.78226195, 0.77633413, 0.76646049, 0.77123849, 0.77799755])

###Write your observations of the above tutorial here.**(5%)**

**Observations:**

In this tutorial we have covered cross validation techniques using an income evaluation dataset with predicted value as income divided into 2 ranges (<= and > 50k), which makes it a classification problem.

In EDA we checked for null values and replaced them with 'missing' if found any.

I have learnt how to divide columns based on datatype (numeric and non-numeric) which is important to implement encoding and scaling for relevant features.

One hot encoding and scaling (robust) has also been used in this tutorial, which is important to massage data and make it ready for ML models to learn these features.

This tutorial covers different validation techniques including k-fold, stratified k-fold, repeated k-fold and leave one out cross validation.

Classifier which we have used here is, random forest in a pipeline setup.

In the first set, we have used k-fold cross validation with 5 splits on our dataset, followed by OHE on categorical columns and robust scaling on numerical columns. Then mentioned column transformation and random forest model were used in a pipelined setup for each split (total 5) producing an average accuracy score of 85%.

In the next set, we have used Stratified k-fold cross validation which is similar to k-fold except equal distribution of observations from all predicted classes. For example, in this case we have 2 classes which we need to predict, <= and > 50k, so we consider an equal ratio of samples from both the classes for training and testing using CV. Rest of the setup is the same with random forest classifier generating around 85% accuracy score.

Third setup covers leave one out cross validation and clearly conveys how costly this technique is w.r.t. to computational time. Where k-fold takes around 7 seconds for the whole dataset, loocv takes more than 3 seconds for only 100 rows.

Next section talks about Repeated k-fold cv which is an extension to k-fold using number of splits as well as having a parameter doing repetition on those splits (n_repeats i.e. 5). In this case 5 splits with 5 repeats generating 25 test scenarios followed by the same ML model generating a similar accuracy score.

In the last set, we use k-fold cross validation on the training set followed by testing on a different test set, hence, using all three splits train, validation and test. First we split the dataset in train and test, followed by k-fold on the training set. Using the same pipe with ohe and scaling for column transformation and random forest model for classification with accuracy as measure. Also, I tried other scoring metrics like roc_auc and f1_macro.


###**Task-1(40%)**
#####Perform cross validation on the tutorial given using the dataset attached.After you are successful in executing and fixing the issues with the current version use dataset of your choice and execute it again.

In [ ]:
!gdown '1zweAahm7OUCQ2y8pgwc5FrGqW9oV6Tbx'

Downloading...
From: https://drive.google.com/uc?id=1zweAahm7OUCQ2y8pgwc5FrGqW9oV6Tbx
To: /content/healthcare-dataset-stroke-data.csv
100% 317k/317k [00:00<00:00, 17.1MB/s]


In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, LeaveOneOut,\
RepeatedKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('healthcare-dataset-stroke-data.csv', na_values=' ?')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
df.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2, weights="uniform")
df['bmi'] = imputer.fit_transform(df[['bmi']])  # impute missing bmi values

In [ ]:
df.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [ ]:
X = df.drop('stroke', axis=1)
y = df['stroke']

print(f'X.shape: {X.shape}')

y.value_counts()

X.shape: (5110, 11)


0    4861
1     249
Name: stroke, dtype: int64

In [ ]:
###1. KFold Cross Validation
kf = KFold(n_splits=5)

print(f'Count in each split: {5110/5}')
print(f'Count in n-1 splits: {(5110/5)*4}\n\n')

i = 1
for train_set, test_set in kf.split(X=X):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print("-------------------------")
    i += 1

Count in each split: 1022.0
Count in n-1 splits: 4088.0


iteration  1
[1022 1023 1024 ... 5107 5108 5109]  having : 4088
[   0    1    2 ... 1019 1020 1021]  having : 1022
-------------------------
iteration  2
[   0    1    2 ... 5107 5108 5109]  having : 4088
[1022 1023 1024 ... 2041 2042 2043]  having : 1022
-------------------------
iteration  3
[   0    1    2 ... 5107 5108 5109]  having : 4088
[2044 2045 2046 ... 3063 3064 3065]  having : 1022
-------------------------
iteration  4
[   0    1    2 ... 5107 5108 5109]  having : 4088
[3066 3067 3068 ... 4085 4086 4087]  having : 1022
-------------------------
iteration  5
[   0    1    2 ... 4085 4086 4087]  having : 4088
[4088 4089 4090 ... 5107 5108 5109]  having : 1022
-------------------------


In [ ]:
num_cols = X.select_dtypes(include=np.number).columns
print(f'Numeric Columns: \n{num_cols}\n\n')

cat_cols = X.select_dtypes(exclude=np.number).columns
print(f'Categorical Columns: \n{cat_cols}\n\n')


ct = ColumnTransformer([
    ('rob', RobustScaler(), num_cols),
    ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_cols)
])

Numeric Columns: 
Index(['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi'],
      dtype='object')


Categorical Columns: 
Index(['gender', 'ever_married', 'work_type', 'Residence_type',
       'smoking_status'],
      dtype='object')




In [ ]:
pipe = Pipeline([
    ('ct_step', ct),
    ('model', RandomForestClassifier(n_estimators=10, random_state=0))
])

X.loc[1022:].head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
1022,60047,Male,22.0,0,0,No,Private,Rural,58.38,36.0,never smoked
1023,16542,Female,60.0,0,0,Yes,Govt_job,Urban,86.34,22.1,never smoked
1024,18805,Male,39.0,0,0,Yes,Private,Urban,95.44,38.4,never smoked
1025,17869,Female,53.0,0,0,Yes,Private,Urban,94.78,30.1,Unknown
1026,6793,Female,55.0,0,0,Yes,Private,Rural,109.59,26.2,formerly smoked


In [ ]:
X

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked
...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,28.893237,never smoked
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,formerly smoked


In [ ]:
scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1


print(f'\n\nScores: \n{np.array(scores)}\n\n')


print(f'Mean of Scores: \n{np.array(scores).mean()}\n\n')


print(f'Standard Deviation of Scores: \n{np.array(scores).std()}\n\n')


iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


Scores: 
[0.75636008 0.99706458 0.99412916 0.99217221 0.99315068]


Mean of Scores: 
0.9465753424657535


Standard Deviation of Scores: 
0.09512172429078737




In [ ]:
###2. Stratified KFold Cross Validation
y.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [ ]:
print(f'Count of class 1 in each split: {y.value_counts()[1]/5}')
print(f'Count of class 1 in n-1 splits: {(y.value_counts()[1]/5)*4}\n')

print(f'Count of class 0 in each split: {y.value_counts()[0]/5}')
print(f'Count of class 0 in n-1 splits: {(y.value_counts()[0]/5)*4}')

Count of class 1 in each split: 49.8
Count of class 1 in n-1 splits: 199.2

Count of class 0 in each split: 972.2
Count of class 0 in n-1 splits: 3888.8


In [ ]:
skf = StratifiedKFold(n_splits=5)

scores_skf = []
i = 1
for train_set, test_set in skf.split(X, y):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores_skf.append(sco)
    print("iteration ", i)
    i += 1


iteration  1
iteration  2
iteration  3
iteration  4
iteration  5


In [ ]:
scores_skf

[0.9481409001956947,
 0.9500978473581213,
 0.9432485322896281,
 0.9471624266144814,
 0.952054794520548]

In [ ]:
i = 1
for train_set, test_set in skf.split(X=X, y=y):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print()
    print("y train counts: \n", y[train_set].value_counts())
    print("y test counts: \n", y[test_set].value_counts())
    print("-------------------------")
    i += 1

iteration  1
[  50   51   52 ... 5107 5108 5109]  having : 4088
[   0    1    2 ... 1218 1219 1220]  having : 1022

y train counts: 
 0    3889
1     199
Name: stroke, dtype: int64
y test counts: 
 0    972
1     50
Name: stroke, dtype: int64
-------------------------
iteration  2
[   0    1    2 ... 5107 5108 5109]  having : 4088
[  50   51   52 ... 2190 2191 2192]  having : 1022

y train counts: 
 0    3889
1     199
Name: stroke, dtype: int64
y test counts: 
 0    972
1     50
Name: stroke, dtype: int64
-------------------------
iteration  3
[   0    1    2 ... 5107 5108 5109]  having : 4088
[ 100  101  102 ... 3162 3163 3164]  having : 1022

y train counts: 
 0    3889
1     199
Name: stroke, dtype: int64
y test counts: 
 0    972
1     50
Name: stroke, dtype: int64
-------------------------
iteration  4
[   0    1    2 ... 5107 5108 5109]  having : 4088
[ 150  151  152 ... 4134 4135 4136]  having : 1022

y train counts: 
 0    3889
1     199
Name: stroke, dtype: int64
y test count

In [ ]:
result_kf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=5)
result_kf

array([0.9481409 , 0.95009785, 0.94324853, 0.94716243, 0.95205479])

In [ ]:
###3. Leave one out Cross Validation

start = time.time()
result_kf10 = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=KFold(n_splits=10))
result_kf10
print("time taken: ", time.time()-start)

time taken:  0.7834956645965576


In [ ]:
result_kf10

array([0.51272016, 0.99608611, 0.99021526, 1.        , 0.99217221,
       0.99608611, 0.99217221, 0.99804305, 0.99412916, 1.        ])

In [ ]:
start = time.time()
result_loocv = cross_val_score(estimator=pipe, X=X.head(100), y=y.head(100),
                               scoring='accuracy', cv=LeaveOneOut())
print("time taken: ", time.time()-start)

time taken:  3.650752544403076


In [ ]:
((X.shape[0]/100)*8)/60

6.8133333333333335

In [ ]:
result_loocv

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
result_loocv.mean()

1.0

In [ ]:
###4. Repeated KFold Cross Validation

start = time.time()
result_rkf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy',
                              cv=RepeatedKFold(n_splits=5, n_repeats=5))
result_rkf
print("time taken: ", time.time()-start)



time taken:  2.65730357170105


In [ ]:
result_rkf

array([0.95401174, 0.94520548, 0.94520548, 0.94520548, 0.95107632,
       0.94227006, 0.95303327, 0.94911937, 0.96086106, 0.94129159,
       0.94911937, 0.94324853, 0.95009785, 0.94911937, 0.94911937,
       0.94618395, 0.95205479, 0.95009785, 0.94520548, 0.95303327,
       0.94911937, 0.9481409 , 0.95401174, 0.95205479, 0.94031311])

In [ ]:
###5. Performing Cross validation on a training set, and then verifying our results on a seperate testset to generalize our results.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

start = time.time()
result_tts = cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='accuracy', cv=KFold(n_splits=5))
print("time taken: ", time.time()-start)

time taken:  0.3511331081390381


In [ ]:
result_tts

array([0.95843521, 0.94987775, 0.95354523, 0.94614443, 0.94492044])

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('ct_step',
                 ColumnTransformer(transformers=[('rob', RobustScaler(),
                                                  Index(['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level',
       'bmi'],
      dtype='object')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['gender', 'ever_married', 'work_type', 'Residence_type',
       'smoking_status'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(n_estimators=10, random_state=0))])

In [ ]:
pipe.score(X_test, y_test)

0.9452054794520548

In [ ]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='f1_macro', cv=KFold(n_splits=5))


array([0.48938826, 0.48714734, 0.51309524, 0.48616352, 0.52662074])

In [ ]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='roc_auc', cv=KFold(n_splits=5))

array([0.67362436, 0.72774743, 0.75544174, 0.73926641, 0.71053541])

**Observations:**

I have executed tutorial on a different dataset from the healthcare domain using features to predict stroke (1/0 classification).

One additional step added during EDA is KNN Imputer for null values in the BMI feature. As BMI is not a categorical feature, we can not replace null values with 'missing', hence using KNN Imputer with 2 neighbors. Rest steps are as same as tutorial, practicing k-fold, stratified, repeated and leave one out cross validation techniques with random forest classifier ML model.


###Tutorial

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
iris=load_iris()
X=iris.data
Y=iris.target
# print("Size of Dataset {}".format(len(X)))
# logreg=LogisticRegression()
# x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)
# logreg.fit(x_train,y_train)
# predict=logreg.predict(x_test)
# print("Accuracy score on training set is {}".format(accuracy_score(logreg.predict(x_train),y_train)))
# print("Accuracy score on test set is {}".format(accuracy_score(predict,y_test)))

####Monte Carlo Cross Validation

In [ ]:
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg=LogisticRegression(max_iter=200)
shuffle_split=ShuffleSplit(test_size=0.3,train_size=0.5,n_splits=10)

In [ ]:
scores=cross_val_score(logreg,iris.data,iris.target,cv=shuffle_split)

In [ ]:
print("cross Validation scores: {}\n\n".format(scores))
print("Average Cross Validation score :{}".format(scores.mean()))

cross Validation scores: [0.95555556 0.97777778 1.         0.93333333 0.97777778 1.
 0.93333333 0.97777778 0.97777778 0.93333333]


Average Cross Validation score :0.9666666666666666


**Observations:**

In this tutorial we have used iris dataset from sklearn performing logistic regression on features using shuffle split i.e. Random permutation cross-validator from the same sklearn library as a cross validation technique, followed by a cross validation score to measure the performance.

####Time Series Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
time_series = TimeSeriesSplit()
print(time_series)
print('\n\n')
for train_index, test_index in time_series.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)



TRAIN: [0] TEST: [1]
TRAIN: [0 1] TEST: [2]
TRAIN: [0 1 2] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]
TRAIN: [0 1 2 3 4] TEST: [5]


**Observations:**

Above tutorial covers the time-series cross validation part. As order is really important in time series based predictions, we can not simply split our dataset in train and test sets, we must follow the order. Hence, for time series cross validation we have to follow different techniques where we can choose the number of splits and test sizes as well, however, they must be in a sequence which can be seen in above output.

###**Task-2(25%)**
###Question-2(a)  **(15)%**
####Explain the types of cross validation techniques. Also write the pros and cons of each validation technique in detail.(7 types).

There are multiple techniques available for cross validation and many of those we have covered in this exercise like k-fold, stratified k-fold, repeated k-fold, leave one out, shuffle split and time series cross validation.

The purpose of using cross validation is to assess the performance of the model, how well it generalizes data points for better prediction on unseen data, and also to reduce overfitting and train hyperparameters.

These techniques overall divide the training set in small chunks and use some for training and others for validation.

We can simply divide the dataset into 2 parts, one for training and another for validation, that's holdout cross validation.
Instead of 2, we can divide into n parts, let's say 5 parts where we use n-1 (4) for training and 1 for validation, that's k-fold cross validation.

In case we want to maintain the distribution of classes in target, we can use stratified k-fold CV, also can repeat k-folds multiple times with random splits which is repeat k-fold CV.

In a special case of k-fold, if we only use one data point for validation and rest for training, that's Leave One Out cross validation (LOOCV). Instead of exactly one, we can leave p samples also out for validation, that's Leave P out cross validation.

We can also randomly shuffle and split on any defined portion of the dataset, which is shuffle split CV.

As mentioned earlier, we need to handle time series differently hence, we have time series cross validation technique.

When to use which technique, as each has its own pros and cons associated, highly depends on the dataset. Few of the advantages and disadvantages are mentioned:

Hold out CV is easy to implement and has less cost, but might not produce better performance and also may result in high variance due to randomness during the split.

k-fold is really helpful as it uses multiple folds and tries to generalize using average of all folds to reduce random impact, however costly for large datasets and won't help with time series.


Stratified k-fold maintains the target class and is really helpful for imbalanced datasets, however time complexity is worse than k-fold due to additional computation. Similar issues for repeated k-fold CV.


LOOCV produces better results for small datasets with low bais, but as the dataset grows, computational cost increases dramatically due to the nature of implementation. Also, as we are leaving only one data point for validation, outliers can heavily impact the performance.

Shuffle split is faster for larger datasets, but random splits can impact the resulting performance.

Time series cross validation helps with time series based datasets as order is important there, however, it requires extra caution for implementation and might not work for some datasets without some tuning.

###Question-2(b)  **(10%)**
####Provide some examples of real-world problems that require cross validation techniques?

In the real world, cross validation helps to generalize the training of models which results in better model performance while predicting unseen data points. In both the cases where the dataset is large or small, one or the other technique will be of real use.

For example, if the dataset is imbalanced, mostly **medical diagnosis** where majority points belong to one specific class. I have used a stroke prediction dataset in this exercise where the majority (92%) data points belong to non-stroke or class 0 while learning class-1 or stroke patient is really important. k-fold or stratified k-fold comes really handy here and the model learns well with CV's help.

For large datasets, with the help of cross validation, we can get the indication of performance of the model at the training phase itself. Hence, one needs not to wait till the end for testing. Large datasets like the **image classification** task might take a long time to complete and without cross validation we need to wait till the end for checking the model performance. Using cross validation here can save lots of iterations and time.

In each industry or market segment, cross validation can help learning datasets better which can save computational cost, time and can derive faster less biased results.


###**Task-3(30%)**
###Question-3(a)
Consider a dataset with 4 samples. Perform KFold, repeated KFold and LOO Cross validations.

In [ ]:
#Write your code here

In [ ]:
from numpy import array
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut

X = np.array([1, 2, 3, 4])

##KFold Cross Validation
print('KFold Cross Validation:')
kf = KFold(n_splits=4, shuffle=False, random_state=None)

i = 1
for train_set, test_set in kf.split(X=X):
  print("iteration ", i)
  print('train: %s, test: %s' % (X[train_set], X[test_set]))
  i+=1






##Repeated KFold Cross Validation
X = np.array([1, 2, 3, 4])
print('\n\nRepeated KFold Cross Validation:')

rkf = RepeatedKFold(n_splits=4, n_repeats=2, random_state=None)
print(f'#Splits: {rkf.get_n_splits(X)}')

print(rkf)
#RepeatedKFold(n_repeats=2, n_splits=2, random_state=2652124)

i = 1
for train_set, test_set in rkf.split(X=X):
  print("iteration ", i)
  print('train: %s, test: %s' % (X[train_set], X[test_set]))
  i+=1



##LOO Cross validations
X = np.array([1, 2, 3, 4])
print('\n\nLOO Cross validations:')
loocv = LeaveOneOut()
print(f'#Splits: {loocv.get_n_splits(X)}')
print(loocv)

i=1
for train_set, test_set in loocv.split(X):
  print(f"iteration {i}:")
  print('train: %s, test: %s' % (X[train_set], X[test_set]))
  i+=1




KFold Cross Validation:
iteration  1
train: [2 3 4], test: [1]
iteration  2
train: [1 3 4], test: [2]
iteration  3
train: [1 2 4], test: [3]
iteration  4
train: [1 2 3], test: [4]


Repeated KFold Cross Validation:
#Splits: 8
RepeatedKFold(n_repeats=2, n_splits=4, random_state=None)
iteration  1
train: [2 3 4], test: [1]
iteration  2
train: [1 3 4], test: [2]
iteration  3
train: [1 2 4], test: [3]
iteration  4
train: [1 2 3], test: [4]
iteration  5
train: [1 2 3], test: [4]
iteration  6
train: [2 3 4], test: [1]
iteration  7
train: [1 3 4], test: [2]
iteration  8
train: [1 2 4], test: [3]


LOO Cross validations:
#Splits: 4
LeaveOneOut()
iteration 1:
train: [2 3 4], test: [1]
iteration 2:
train: [1 3 4], test: [2]
iteration 3:
train: [1 2 4], test: [3]
iteration 4:
train: [1 2 3], test: [4]


###Question-3(b)
Consider a dataset with 6 samples and perform 3 split time-series cross validation

In [ ]:
#Write your code here

In [ ]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

print("Time-series CV with 6 samples and 3 splits:\n")
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])

time_series = TimeSeriesSplit(n_splits=3)
print(time_series)
print('\n')

i=1
for train_index, test_index in time_series.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
    i+=1




Time-series CV with 6 samples and 3 splits:

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)


TRAIN: [0 1 2] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]
TRAIN: [0 1 2 3 4] TEST: [5]


In [ ]:
# Fix test_size to 2 with 12 samples & #splits as 3
X = np.random.randn(12, 2)
y = np.random.randint(0, 2, 12)

print("Time-series CV with 12 samples, 3 splits and a test size of 2:")
time_series = TimeSeriesSplit(n_splits=3, test_size=2)
print(time_series)
print('\n')

i=1
for train_index, test_index in time_series.split(X):
  print(f"iteration {i}:")
  print("TRAIN:", train_index, "TEST:", test_index, '\n')
  i+=1


Time-series CV with 12 samples, 3 splits and a test size of 2:
TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=2)


iteration 1:
TRAIN: [0 1 2 3 4 5] TEST: [6 7] 

iteration 2:
TRAIN: [0 1 2 3 4 5 6 7] TEST: [8 9] 

iteration 3:
TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10 11] 



**Observations:**

In task 3, we covered k-fold, repeated k-fold and leave one out cross validation on a simple dataset of array of values to develop a better and deep understanding with the inner workings of cross validation techniques.

Also touched upon the time-series cross validation with few samples using different numbers of splits and test sizes to check the impact on training samples which will be given to models for learning.
